
# Python project - Hotel


The dataset chosen for the project contains the following information:
* hotels.xlsx: number of vacant rooms and unit cost of each room for 400 hotels;
* guests.xlsx: discount fraction for 4000 potential customers;
* preferences.xlsx: order of hotel preference for each customer.
The program must calculate the allocation of customers at hotels, considering the number of available rooms, the fact that each customer occupies exactly one room, and that each stay lasts only one night. The price paid by the customer is the unit price of the room discounted by the fraction of the discount to which the customer is entitled.

The program must implement four different allocation strategies:
* random: customers are randomly distributed to the rooms until the seats or customers are exhausted;
* customer preference: customers are served in order of reservation (the customer number indicates the order) and are allocated to the hotel based on their preference, until the seats or customers are exhausted;
* price: the places in the hotel are distributed in order of price, starting with the cheapest hotel and following in order of reservation and preference until the places or customers are exhausted;
* availability: places in hotels are distributed in order of room availability, starting with the most roomy hotel and subordinately in order of reservation and preference until places or clients are exhausted.

Finally, the program must present and display a report of the result obtained, showing for each strategy the number of customers accommodated, the number of rooms occupied, the number of different hotels occupied, the total volume of business (total earnings of each hotel), and the degree of customer satisfaction (calculated according to the location of the hotel assigned to them with respect to their preferences).

# Files in the project

* **random_allocation.py**: it implements the random allocation
* **preferences_allocation.py**: it implements the customer preferences allocation
* **price_allocation.py**: it implements the price allocation
* **availability_allocation.py**: it implements the availability allocation
* **utils.py**: it defines the function 'satisfaction' which calculates the satisfaction percentage
* **main.py**: it implements the allocations, showing the output
* **HotelProject_presentation.ipynb**: shows the output in a jupyter notebook

# Importing data

In [61]:
import pandas as pd
import openpyxl
import numpy as np
from utils import satisfaction


In [62]:
guests= pd.read_excel(r"C:\Users\lejda\Desktop\coding - Python\guests.xlsx")
guests

,Unnamed: 0,guest,discount
0,0,guest_1,0.09
1,1,guest_2,0.00
2,2,guest_3,0.07
3,3,guest_4,0.00
4,4,guest_5,0.10
...,...,...,...
3995,3995,guest_3996,0.00
3996,3996,guest_3997,0.15
3997,3997,guest_3998,0.07
3998,3998,guest_3999,0.08


In [63]:
hotels = pd.read_excel(r"C:\Users\lejda\Desktop\coding - Python\hotels.xlsx")
hotels

,Unnamed: 0,hotel,rooms,price
0,0,hotel_1,13,273
1,1,hotel_2,18,92
2,2,hotel_3,12,141
3,3,hotel_4,18,157
4,4,hotel_5,7,298
...,...,...,...,...
395,395,hotel_396,5,212
396,396,hotel_397,12,68
397,397,hotel_398,14,102
398,398,hotel_399,16,284


In [64]:
preferences = pd.read_excel(r"C:\Users\lejda\Desktop\coding - Python\preferences.xlsx")
preferences

,Unnamed: 0,guest,hotel,priority
0,0,guest_1,hotel_168,1
1,1,guest_1,hotel_207,2
2,2,guest_1,hotel_222,3
3,3,guest_1,hotel_124,4
4,4,guest_1,hotel_223,5
...,...,...,...,...
99528,99528,guest_4000,hotel_123,28
99529,99529,guest_4000,hotel_396,29
99530,99530,guest_4000,hotel_161,30
99531,99531,guest_4000,hotel_330,31


# Random Allocation

In [113]:
class HotelAllocation:
    def __init__(self, hotels, guests, preferences):
        """
        Initialize the HotelAllocation.

        Parameters:
        - hotels (pd.DataFrame): DataFrame containing information about hotels.
        - guests (pd.DataFrame): DataFrame containing information about guests.
        """
        #we use copies to avoid modifying the original dataframes
        self.hotels = hotels.copy()
        self.guests = guests.copy()
        self.preferences = preferences.copy()
        
    def accomodate_single_guest(self, guest_row):
        pass

    def accomodate_guests(self):
        allocations = []

        for _, guest_row in self.guests.iterrows(): #iterrows returns a series for each row
            allocation_entry = self.accomodate_single_guest(guest_row) #allocate the current guest to a random avaiable hotel
            if allocation_entry is not None:
                allocations.append(allocation_entry)

        allocation_df = pd.DataFrame(allocations, columns=['guest_id', 'hotel_id', 'paid_price'])
        return allocation_df

class RandomHotelAllocation(HotelAllocation):
    """ 
    Allocate a guest to a random available hotel based on availability.

    Parameters:
    - guest_row (pd.Series): A row from the guests DataFrame.

    Returns:
    - dict: Allocation information with keys 'guest_id', 'hotel_id', and 'paid_price'.
    """
    def calculate_satisfaction_percentage(self, guest_id, hotel_id):
        guest_preferences = self.preferences[self.preferences['guest'] == guest_id].reset_index() #filter preferences for the given guest
        if guest_preferences.empty:
            return 100  # No preferences, 100% satisfaction

        index_of_preference = (guest_preferences['hotel'] == hotel_id).idxmax() # Find the index of the allocated hotel in the guest's         preferences
        satisfaction = round(((len(guest_preferences) - index_of_preference) / len(guest_preferences)) * 100)
        return satisfaction if satisfaction >= 0 else 0
    
    def accomodate_single_guest(self, guest_row):
        available_hotels = self.hotels[self.hotels['rooms'] > 0]
        if available_hotels.empty:
            return None

        random_hotel_id = np.random.choice(available_hotels.index) #we randomly select a hotel with available rooms
        self.hotels.loc[random_hotel_id, 'rooms'] -= 1 #reduce the number of avaiable rooms in that hotel by 1
        
        #we calculate the discounted price that a guest will pay for staying in the randomly allocated hotel,
        #taking into account both the original price of the hotel and the guest's discount.
        price_paid = round(available_hotels.loc[random_hotel_id, 'price'] * (1 - guest_row['discount']), 2)
        satisfaction = self.calculate_satisfaction_percentage(guest_id, random_hotel_id, preferences)

        return {'guest_id': guest_row.name, 'hotel_id': random_hotel_id, 'paid_price': price_paid, 'satisfaction': satisfaction}


# Preference Allocation

In [114]:
class PreferencesAllocator:
    def __init__(self, hotels, guests, preferences):
        """Initialize the AvailabilityBasedAllocator.

            Parameters:
          - hotels (pd.DataFrame): DataFrame containing information about hotels.
          - guests (pd.DataFrame): DataFrame containing information about guests.
          - preferences (pd.DataFrame): DataFrame containing guest preferences.
        """
        #we use copies to avoid modifying the original dataframes   
        self.hotels = hotels.copy()
        self.guests = guests.copy()
        self.preferences = preferences.copy()
        
    def calculate_satisfaction_percentage(self, guest_id, hotel_id):
        guest_preferences = self.preferences[self.preferences['guest'] == guest_id].reset_index() #filter preferences for the given guest
        if guest_preferences.empty:
            return 100  # No preferences, 100% satisfaction

        index_of_preference = (guest_preferences['hotel'] == hotel_id).idxmax() # Find the index of the allocated hotel in the guest's         preferences
        satisfaction = round(((len(guest_preferences) - index_of_preference) / len(guest_preferences)) * 100)
        return satisfaction if satisfaction >= 0 else 0
        

    def allocate_and_calculate(self):
        """Allocate guests to their preferred hotels, calculate paid price and satisfaction.

        Returns:
        - pd.DataFrame: DataFrame containing allocation information for each guest,
          including guest ID, hotel ID, satisfaction, and paid price.
        """
        allocation_list = []

        for guest_id, guest_row in self.guests.iterrows():
            #the code is iterating through the guests in the guests DataFrame and, for each guest,
            #extracting the preferred hotels from the preferences DataFrame based on their ID.
            guest_preferred_hotels = self.preferences[self.preferences['guest'] == f'guest_{guest_id}']['hotel']

            for _, preferred_hotel_id in guest_preferred_hotels.items():
                #we extract a numerical index from hotel_id, removing the prefix "hotel_", subtracting 1 to the remain number
                #and covert it into an integer to get the hotel_id number
                hotel_index = int(preferred_hotel_id.lstrip('hotel_')) - 1
                preferred_hotel_row = self.hotels.loc[hotel_index]

                if preferred_hotel_row['rooms'] > 0:
                    self.hotels.loc[hotel_index, 'rooms'] -= 1

                    paid_price_coefficient = 1 - guest_row['discount']
                    paid_price = preferred_hotel_row['price'] * paid_price_coefficient

                    satisfaction = self.calculate_satisfaction_percentage(guest_id, preferred_hotel_id, self.preferences)

                    allocation_entry = [guest_id, preferred_hotel_id, satisfaction, paid_price]
                    allocation_list.append(allocation_entry)

                    break

        return pd.DataFrame(allocation_list, columns=['guest_id', 'hotel_id', 'satisfaction', 'paid_price'])
       

# Price allocation

In [115]:
class PriceBasedAllocator:
    def __init__(self, hotels, guests, preferences):
        """Initialize the AvailabilityBasedAllocator.

            Parameters:
          - hotels (pd.DataFrame): DataFrame containing information about hotels.
          - guests (pd.DataFrame): DataFrame containing information about guests.
          - preferences (pd.DataFrame): DataFrame containing guest preferences.
        """
        #we use copies to avoid modifying the original dataframes   
        self.hotels = hotels.copy()
        self.guests = guests.copy()
        self.preferences = preferences.copy()
    
    def calculate_satisfaction_percentage(self, guest_id, hotel_id):
        guest_preferences = self.preferences[self.preferences['guest'] == guest_id].reset_index() #filter preferences for the given guest
        if guest_preferences.empty:
            return 100  # No preferences, 100% satisfaction

        index_of_preference = (guest_preferences['hotel'] == hotel_id).idxmax() # Find the index of the allocated hotel in the guest's         preferences
        satisfaction = round(((len(guest_preferences) - index_of_preference) / len(guest_preferences)) * 100)
        return satisfaction if satisfaction >= 0 else 0
        
    def can_allocate_to_hotel(self, hotel_row, guest_id):
        """
        Check if a guest can be allocated to a hotel.

        Parameters:
        - hotel_row (pd.Series): Row of the hotel DataFrame.
        - guest_id: ID of the guest.

        Returns:
        - bool: True if allocation is possible, False otherwise.
        """
        #we ensure that there are available rooms in the hotel and if the guest has preferences for that hotel
        return hotel_row['rooms'] > 0 and f'guest_{guest_id}' in self.preferences['guest'].values

    def calculate_paid_price(self, row):
        return row['price'] * (1 - row['discount'])

    def allocate_and_calculate(self):
        """
        Allocate guests to hotels based on price, calculate satisfaction and paid price.

        Returns:
        - pd.DataFrame: DataFrame containing allocation information.
        """
        allocation_list = []
        #we merge the three dataframes and we sort the values based on price in ascending order
        sorted_hotels = self.preferences.merge(self.hotels, on=['hotel']).merge(self.guests).sort_values(by='price')

        for group_key, group in sorted_hotels.groupby('hotel', sort=False): #we group the rows of sorted_hotels dataframe based on the         hotel column 
            for id, row in group.iterrows():
                if group.iloc[0]['rooms'] == 0: #we check if the number of available rooms in the first row of the current group is                      equal to zero
                    break
                group['rooms'] -= 1 #we decrement the number of rooms for the current hotel group by 1
                paid_price = self.calculate_paid_price(row)
                satisfaction = self.calculate_satisfaction_percentage(row['guest'], row['hotel'], preferences)
                allocation_entry = [row['guest'], row['hotel'], satisfaction, paid_price]
                allocation_list.append(allocation_entry)
        return pd.DataFrame(allocation_list, columns=['guest_id', 'hotel_id', 'satisfaction', 'paid_price'])

# Availability Allocation

In [116]:
class AvailabilityBasedAllocator:
    def __init__(self, hotels, guests, preferences):
        """
        Initialize the AvailabilityBasedAllocator.

        Parameters:
        - hotels (pd.DataFrame): DataFrame containing information about hotels.
        - guests (pd.DataFrame): DataFrame containing information about guests.
        - preferences (pd.DataFrame): DataFrame containing guest preferences.
        """
        # Use copies to avoid modifying the original DataFrames
        self.hotels = hotels.copy()
        self.guests = guests.copy()
        self.preferences = preferences.copy()
        
    def calculate_satisfaction_percentage(self, guest_id, hotel_id):
        guest_preferences = self.preferences[self.preferences['guest'] == guest_id].reset_index() #filter preferences for the given guest
        if guest_preferences.empty:
            return 100  # No preferences, 100% satisfaction

        index_of_preference = (guest_preferences['hotel'] == hotel_id).idxmax() # Find the index of the allocated hotel in the guest's         preferences
        satisfaction = round(((len(guest_preferences) - index_of_preference) / len(guest_preferences)) * 100)
        return satisfaction if satisfaction >= 0 else 0

    def allocate_and_calculate(self):
        """
        Allocate guests to hotels based on available rooms, calculate satisfaction, and paid price.

        Returns:
        - pd.DataFrame: DataFrame containing allocation information.
        """
        allocation_list = []

        # Sort hotels based on available rooms (desc order)
        sorted_hotels = self.hotels.sort_values(by='rooms', ascending=False)

        for _,hotel_row in sorted_hotels.iterrows():
            allocated_guests = set()  #iniziales an empty set and track guests already allocated to the current hotel
            for guest_row in self.guests.iterrows():
                guest_id = guest_row['guest_id']

                # Check if the guest can be allocated to the current hotel
                if can_allocate_to_hotel(hotel_row, guest_id, self.preferences) and guest_id not in allocated_guests:
                    paid_price = hotel_row['price'] * (1 - guest_row['discount'])
                    satisfaction = self.calculate_satisfaction_percentage(guest_id, hotel_row.name, self.preferences)
                    allocation_entry = [guest_id, hotel_row.name, satisfaction, paid_price]
                    allocation_list.append(allocation_entry)

                    # Update the set of allocated guests for the current hotel
                    allocated_guests.add(guest_id)

        return pd.DataFrame(allocation_list, columns=['guest_id', 'hotel_id', 'satisfaction', 'paid_price'])

# Visualization

In [110]:
import matplotlib.pyplot as plt

In [117]:
def visualization(allocation):
    fig, (ax1, ax2) = plt.subplots(2, 2, figsize=(15, 10))
    #calculate total earnings and mean earnings for each hotel
    paid_price_by_hotel_sum = allocation.groupby('hotel_id')['paid_price'].sum()
    paid_price_by_hotel_mean = allocation.groupby('hotel_id')['paid_price'].mean()
    
    # Plot total earnings
    paid_price_by_hotel_sum.plot(kind='bar', ax=ax1, title='Total Earnings by Hotel')
    ax1.set_xlabel('Hotel ID')
    ax1.set_ylabel('Paid Price Sum')

    # Plot mean earnings
    paid_price_by_hotel_mean.plot(kind='bar', ax=ax2, title='Mean Earnings by Hotel')
    ax2.set_xlabel('Hotel ID')
    ax2.set_ylabel('Paid Price Mean')
    
    #graph 2: boxplot of the mean satisfaction percentage per guest
    satisfaction_by_guest = allocation.groupby('guest_id')['satisfaction_percentage'].mean()
    ax2.boxplot(satisfaction_by_guest, vert=False)
    ax2.set_title('Boxplot of Mean Satisfaction Percentage by Guest')
    ax2.set_xlabel('Satisfaction Percentage')

    plt.show()

# Output

In [118]:
def main():
    guests= pd.read_excel(r"C:\Users\lejda\Desktop\coding - Python\guests.xlsx")
    hotels = pd.read_excel(r"C:\Users\lejda\Desktop\coding - Python\hotels.xlsx")
    preferences = pd.read_excel(r"C:\Users\lejda\Desktop\coding - Python\preferences.xlsx")
    
    random_allocator = RandomHotelAllocation(hotels, guests, preferences)
    r_allocation = random_allocator.accomodate_guests()
    # Print and visualize random allocation
    print("Random Allocation:")
    print(r_allocation)
    visualization(r_allocation)
    
    preferences_allocator = PreferencesAllocator(hotels, guests, preferences)
    p_allocation = preferences_allocator.allocate_and_calculate()
    # Print and visualize preferences allocation
    print("Preferences Allocation:")
    print(p_allocation)
    visualization(p_allocation)
    
    price_allocator = PriceBasedAllocator(hotels, guests, preferences)
    price_allocation = price_allocator.allocate_and_calculate()
    # Print and visualize price-based allocation
    print("Price-Based Allocation:")
    print(price_allocation)
    visualization(price_allocation)
    
    availability_allocator = AvailabilityBasedAllocator(hotels, guests, preferences)
    availability_allocation = availability_allocator.allocate_and_calculate()
    # Print and visualize availability-based allocation
    print("Availability-Based Allocation:")
    print(availability_allocation)
    visualization(availability_allocation)

if __name__ == "__main__":
    main()

NameError: name 'guest_id' is not defined

In [ ]:
main()